In [ ]:
import http.client
import mimetypes
import requests
import json
import csv
import pandas as pd
from datetime import date
from datetime import timedelta
from datetime import datetime
import time
from urllib.parse import urlencode
import telegram

In [ ]:
def gettoken(apitoken):
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    url = "apiconnect.angelbroking.com/rest/auth/angelbroking/user/v1/loginByPassword"
    payload = {"clientcode":"A594088",
                "password":"Angel.jav123"}
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-UserType': 'USER',
        'X-SourceID': 'WEB',
        'X-ClientLocalIP': '192.168.1.11',
        'X-ClientPublicIP': '114.143.211.242',
        'X-MACAddress': '38:f9:d3:49:cd:8e',
        'X-PrivateKey': apitoken
      }

    conn.request("POST", "/rest/auth/angelbroking/user/v1/loginByPassword",json.dumps(payload),headers)

    res = conn.getresponse()
    data = res.read()
    #print(data)
    df = pd.read_json(data)
    token = 'Bearer ' + df.loc['jwtToken', 'data']
    return token

In [ ]:
def getltp(symtoken,token,company):
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    payload = { "exchange":"NSE",    "tradingsymbol": company, "symboltoken": symtoken}

    headers = {
      'Authorization': token,
      'Content-Type': 'application/json',
      'Accept': 'application/json',
      'X-UserType': 'USER',
      'X-SourceID': 'WEB',
      'X-ClientLocalIP': '192.168.1.11',
      'X-ClientPublicIP': '114.143.211.242',
      'X-MACAddress': '38:f9:d3:49:cd:8e',
      'X-PrivateKey': 'tXGeTvju'
    }
    conn.request("POST","/rest/secure/angelbroking/order/v1/getLtpData", json.dumps(payload), headers)

    res = conn.getresponse()
    data = res.read()
    print(data.decode("utf-8"))
    print()
    return data

In [ ]:
def getcandledata(token, symtoken, fromdate, todate):
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    url = "apiconnect.angelbroking.com/rest/auth/angelbroking/user/v1/loginByPassword"
    payload = {
         "exchange": "NSE",
         "symboltoken": symtoken,
         "interval": "ONE_MINUTE",
         "fromdate": fromdate, #format: "2021-02-10 09:15"
         "todate": todate, # format: "2021-03-10 15:30"
    }

    headers = {
      'X-PrivateKey': 'GyFrQxvG',
      'Accept': 'application/json',
      'X-SourceID': 'WEB',
      'Authorization': token,
      'X-ClientLocalIP': '192.168.1.11',
      'X-ClientPublicIP': '114.143.211.242',
      'X-MACAddress': '38:f9:d3:49:cd:8e',
      'X-UserType': 'USER',
      'Accept': 'application/json',
      'X-SourceID': 'WEB',
      'Content-Type': 'application/json'
    }

    conn.request("POST", "/rest/secure/angelbroking/historical/v1/getCandleData", json.dumps(payload), headers)
    res = conn.getresponse()
    data = res.read()
    #print(data.decode("utf-8"))
    return data

In [ ]:
def getsymtoken(symbol):
    ins = pd.read_json('/Users/abhinahar/Downloads/tradeable_instruments.json')
    symtoken = ins.loc[ins[ins['symbol'] == symbol+'-EQ'].index, 'token'].item()
    return symtoken

In [ ]:
def main(comp):
    ltptoken = gettoken('tXGeTvju')
    candletoken = gettoken('GyFrQxvG')
    company = comp
    companyeq = company + '-EQ'
    todate = str(datetime.now())[:10]
    while(tradestatus == False):
        if ((datetime.now().minute >= 17) and (datetime.now().minute <= 18)):
            taketrade(todate + ' 09:15', todate + ' 09:16', 18, candletoken, ltptoken, company, companyeq)
        elif ((datetime.now().minute >= 19) and (datetime.now().minute <= 20)):
            taketrade(todate + ' 09:17', todate + ' 09:18', 20, candletoken, ltptoken, company, companyeq) 
        elif ((datetime.now().minute >= 21) and (datetime.now().minute <= 22)):
            taketrade(todate + ' 09:19', todate + ' 09:20', 22, candletoken, ltptoken, company, companyeq)        
        elif ((datetime.now().minute >= 23) and (datetime.now().minute <= 24)):
            taketrade(todate + ' 09:21', todate + ' 09:22', 24, candletoken, ltptoken, company, companyeq)        
        elif ((datetime.now().minute >= 25) and (datetime.now().minute <= 26)):
            taketrade(todate + ' 09:23', todate + ' 09:24', 26, candletoken, ltptoken, company, companyeq)        
        elif ((datetime.now().minute >= 27) and (datetime.now().minute <= 28)):
            taketrade(todate + ' 09:25', todate + ' 09:26', 28, candletoken, ltptoken, company, companyeq)        
        elif ((datetime.now().minute >= 29) and (datetime.now().minute <= 30)):
            taketrade(todate + ' 09:27', todate + ' 09:28', 30, candletoken, ltptoken, company, companyeq)

    while(tradestatus == True):
        if datetime.now().minute <= 30:
            LTP = list(pd.read_json(getltp(getsymtoken(company), ltptoken, companyeq))['data'])[4]
            if LTP >= real_result.loc[0,'Entry Price'] * 1.01:
                real_result.loc[0,'Exit Time'] = str(datetime.now())
                real_result.loc[0,'Exit Price'] = LTP
                real_result.loc[0,'Percentage'] = (LTP/real_result.loc[0,'Entry Price']-1) * 100
                print("Position exited at Rs.{} at time {}".format(LTP, datetime.now()))
                break
            elif LTP <= real_result.loc[0,'Entry Price'] * .99:
                real_result.loc[0,'Exit Time'] = str(datetime.now())
                real_result.loc[0,'Exit Price'] = LTP
                real_result.loc[0,'Percentage'] = (LTP/real_result.loc[0,'Entry Price']-1) * 100
                print("Position exited at Rs.{} at time {}".format(LTP, datetime.now()))
                break
            time.sleep(0.135)
        else:
            LTP = list(pd.read_json(getltp(getsymtoken(company), ltptoken, companyeq))['data'])[4]
            real_result.loc[0,'Exit Time'] = str(datetime.now())
            real_result.loc[0,'Exit Price'] = LTP
            real_result.loc[0,'Percentage'] = (LTP/real_result.loc[0,'Entry Price']-1) * 100
            print("Position forcefully exited at Rs.{} at time {}".format(LTP, datetime.now()))
            break
            
            
        
            
            
            

In [ ]:
def taketrade(fromdate, todate, minut, candletoken, ltptoken, company, companyeq):
    try:
        data = getcandledata(candletoken, getsymtoken(company), fromdate, todate)
        tmp = pd.read_json(data)
        a = list(tmp['data'])
        columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        df = pd.DataFrame(a, columns = columns)
        if len(df) == 2:
            high = max(df.loc[0,'high'],df.loc[1,'high'])
            low = min(df.loc[0,'low'],df.loc[1,'low'])
        global tradestatus
        while((datetime.now().minute <= minut)and(len(df) == 2)):
            LTP = list(pd.read_json(getltp(getsymtoken(company), ltptoken, companyeq))['data'])[4]
            if LTP >= high:
                real_result.loc[0,'Position Type'] = "Long"
                real_result.loc[0,'Entry Time'] = str(datetime.now())
                real_result.loc[0,'Entry Price'] = LTP
                tradestatus = True
                print("Position taken at Rs.{} at time {}".format(LTP, datetime.now()))
                bot.sendMessage(chat_id = '-486459146', text="Company: {}\n\nLong Position taken at Rs.{} at time {},".format(company, LTP, datetime.now()))
                break
            elif LTP <= low:
                real_result.loc[0,'Position Type'] = "Short"
                real_result.loc[0,'Entry Time'] = str(datetime.now())
                real_result.loc[0,'Entry Price'] = LTP
                tradestatus = True
                print("Position taken at Rs.{} at time {}".format(LTP, datetime.now()))
                bot.sendMessage(chat_id = '-486459146', text="Company: {}\n\nShort Position taken at Rs.{} at time {},".format(company, LTP, datetime.now()))
                break
            time.sleep(0.135)
        return
    except:
        print("ERROR")
        return
            
                
                
                
                
                
                
                

<h1>Enter Company Details here:<h1>

In [ ]:
colum = ['Postion Type','Entry Time', 'Exit Time', 'Entry Price', 'Exit Price', 'Percentage']
real_result = pd.DataFrame(columns = colum)
tradestatus = False
bot = telegram.Bot(token = '1904162812:AAFJUKftPgt7KLs4aLHBpWV6_qkH5l4WRug')
company = 'APOLLOTYR'
main(company)
real_result.to_csv('/Users/abhinahar/Desktop/{}_TRADE.csv'.format(company))
bot.sendMessage(chat_id = '-486459146', text="{} \n{} Performance\n\n\
Entry Time: {} \nExit Time: {}\nEntry Price: {}\nExit Price: {} \nNet Gain:\
 {}%".format(str(datetime.now())[:10], company, str(real_result.loc[0,'Entry Time'])[11:16],str(real_result.loc[0,'Exit Time'])[11:16], real_result.loc[0,'Entry Price'], real_result.loc[0,'Exit Price'],real_result.loc[0,'Percentage']))